## Testing with flattened CellMap

In [35]:
import os
import numpy as np
import pandas as pd
import config
from source.utils import loadmat
import starfish as sf

In [12]:
# dir_path = os.path.dirname(os.path.realpath(__file__))
roi = config.DEFAULT['roi']
label_image_path = os.path.join('./', 'demo_data', 'CellMap.mat')
label_image = loadmat(os.path.join(label_image_path))
label_image = label_image["CellMap"]

In [51]:
t = label_image>0
pd.DataFrame(label_image)

,0,1,2,3,4,5,6,7,8,9,...,7592,7593,7594,7595,7596,7597,7598,7599,7600,7601
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
df = pd.DataFrame(label_image).unstack()
df

0     0       0
      1       0
      2       0
      3       0
      4       0
      5       0
      6       0
      7       0
      8       0
      9       0
      10      0
      11      0
      12      0
      13      0
      14      0
      15      0
      16      0
      17      0
      18      0
      19      0
      20      0
      21      0
      22      0
      23      0
      24      0
      25      0
      26      0
      27      0
      28      0
      29      0
             ..
7601  5441    0
      5442    0
      5443    0
      5444    0
      5445    0
      5446    0
      5447    0
      5448    0
      5449    0
      5450    0
      5451    0
      5452    0
      5453    0
      5454    0
      5455    0
      5456    0
      5457    0
      5458    0
      5459    0
      5460    0
      5461    0
      5462    0
      5463    0
      5464    0
      5465    0
      5466    0
      5467    0
      5468    0
      5469    0
      5470    0
Length: 41590542, dtype:

In [49]:
df_2 = df[df>0]
cellmap = df_2.reset_index().rename(columns={'level_0': 'y', 'level_1': 'x', 0: 'parent_cell'})

,y,x,parent_cell
0,0,718,1
1,0,719,1
2,0,720,1
3,0,721,1
4,0,722,1
5,0,723,1
6,0,724,1
7,0,725,1
8,0,726,1
9,0,727,1


In [33]:
saFile = os.path.join('./', 'demo_data', 'spots.csv')

In [37]:
sa_df = pd.read_csv(saFile)
sa_df = sa_df.rename(columns={'xc': 'x', 'yc': 'y', 'zc': 'z', 'Gene': 'target'})
sa = sf.types.SpotAttributes(sa_df)
sa.data['x'] = sa.data.x - 1
sa.data['y'] = sa.data.y - 1

In [39]:
sa.data

,index,target,radius,x,y,z
0,0,Cxcl14,NaN,6260.0,13001.0,0
1,1,Plp1,NaN,6149.0,13723.0,0
2,2,Plp1,NaN,6149.0,13733.0,0
3,3,Id2,NaN,6149.0,13980.0,0
4,4,Enpp2,NaN,6149.0,14003.0,0
5,5,Plp1,NaN,6149.0,14013.0,0
6,6,Aldoc,NaN,6149.0,14185.0,0
7,7,Enpp2,NaN,6150.0,13717.0,0
8,8,Plp1,NaN,6150.0,13941.0,0
9,9,Id2,NaN,6150.0,13974.0,0
